<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!
Training dataset size:  60000
Testing dataset size:  10000


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self, dropout_rate):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()
        self.dropout  = torch.nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        x = self.dropout(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 30
criterion = torch.nn.CrossEntropyLoss()
def sparsity_trainer(optimizer, model):

    # reset the model 
    train_loss = list()
    test_loss  = list()
    test_acc   = list()
    final_spareness = list()

    # define activation list 

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        hidden_layer_activation_list = list()

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):

            image, label = image.to(device), label.to(device)

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

        # this conains activations for all epochs 
        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
            # print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            # torch.save(model.state_dict(), "model.dth")

    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return test_acc, sparseness_list

In [ ]:
dropout_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# AdaDelta

In [ ]:
for i in range(9):
    print(f"Entering {i}-th loop")
    model_Adadelta = Model(dropout_list[i])
    print("dropout rate:", model_Adadelta.dropout)
    model_Adadelta.to(device)
    model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
    optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
    Adadelta_test_acc, Adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)
    
    if i == 0:
        f = open("dropout_sparsity_Adadelta.txt", "w")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_Adadelta.txt", "a")
        f.write(str(i)+'\n'+str(Adadelta_test_acc)+'\n'+str(Adadelta_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_Adadelta.txt /content/drive/MyDrive

Entering 0-th loop
dropout rate: Dropout(p=0.1, inplace=False)

Epoch: 1/30, Train Loss: 0.44707491, Test Loss: 0.24905709, Test Accuracy: 0.92670000

Epoch: 2/30, Train Loss: 0.22482164, Test Loss: 0.17467613, Test Accuracy: 0.94620000

Epoch: 3/30, Train Loss: 0.16510881, Test Loss: 0.13607034, Test Accuracy: 0.95730000

Epoch: 4/30, Train Loss: 0.13228526, Test Loss: 0.11752594, Test Accuracy: 0.96240000

Epoch: 5/30, Train Loss: 0.10984081, Test Loss: 0.09784792, Test Accuracy: 0.97070000

Epoch: 6/30, Train Loss: 0.09513158, Test Loss: 0.09220489, Test Accuracy: 0.97190000

Epoch: 7/30, Train Loss: 0.08338319, Test Loss: 0.08704453, Test Accuracy: 0.97400000

Epoch: 8/30, Train Loss: 0.07454653, Test Loss: 0.07903682, Test Accuracy: 0.97490000

Epoch: 9/30, Train Loss: 0.06667648, Test Loss: 0.07378661, Test Accuracy: 0.97660000

Epoch: 10/30, Train Loss: 0.05959624, Test Loss: 0.07139345, Test Accuracy: 0.97670000

Epoch: 11/30, Train Loss: 0.05564461, Test Loss: 0.06994762, Test

# AdaGrad

In [ ]:
for i in range(9):
    print(f"Entering {i}-th loop")
    model_Adagrad = Model(dropout_list[i])
    print("dropout rate:", model_Adagrad.dropout)
    model_Adagrad.to(device)
    model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
    optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
    Adagrad_test_acc, Adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)
    
    if i == 0:
        f = open("dropout_sparsity_Adagrad.txt", "w")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_Adagrad.txt", "a")
        f.write(str(i)+'\n'+str(Adagrad_test_acc)+'\n'+str(Adagrad_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_Adagrad.txt /content/drive/MyDrive

Entering 0-th loop
dropout rate: Dropout(p=0.1, inplace=False)

Epoch: 1/30, Train Loss: 0.25509324, Test Loss: 0.12969932, Test Accuracy: 0.96120000

Epoch: 2/30, Train Loss: 0.11281044, Test Loss: 0.10596037, Test Accuracy: 0.96850000

Epoch: 3/30, Train Loss: 0.08366589, Test Loss: 0.09022687, Test Accuracy: 0.97270000

Epoch: 4/30, Train Loss: 0.06728950, Test Loss: 0.08172638, Test Accuracy: 0.97430000

Epoch: 5/30, Train Loss: 0.05651686, Test Loss: 0.07722635, Test Accuracy: 0.97520000

Epoch: 6/30, Train Loss: 0.04891934, Test Loss: 0.07435574, Test Accuracy: 0.97550000

Epoch: 7/30, Train Loss: 0.04265273, Test Loss: 0.07262670, Test Accuracy: 0.97690000

Epoch: 8/30, Train Loss: 0.03734522, Test Loss: 0.07055816, Test Accuracy: 0.97670000

Epoch: 9/30, Train Loss: 0.03320964, Test Loss: 0.06909148, Test Accuracy: 0.97780000

Epoch: 10/30, Train Loss: 0.02928658, Test Loss: 0.06841787, Test Accuracy: 0.97870000

Epoch: 11/30, Train Loss: 0.02714276, Test Loss: 0.06924440, Test

# SGD

In [ ]:
for i in range(9):
    print(f"Entering {i}-th loop")
    model_SGD = Model(dropout_list[i])
    print("dropout rate:", model_SGD.dropout)
    model_SGD.to(device)
    model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
    optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
    SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD)
    
    if i == 0:
        f = open("dropout_sparsity_SGD.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_SGD.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_SGD.txt /content/drive/MyDrive

Entering 0-th loop
dropout rate: Dropout(p=0.1, inplace=False)

Epoch: 1/30, Train Loss: 0.80111968, Test Loss: 0.37000336, Test Accuracy: 0.89550000

Epoch: 2/30, Train Loss: 0.37227186, Test Loss: 0.31034909, Test Accuracy: 0.91290000

Epoch: 3/30, Train Loss: 0.32832539, Test Loss: 0.28662897, Test Accuracy: 0.91880000

Epoch: 4/30, Train Loss: 0.30296913, Test Loss: 0.26827574, Test Accuracy: 0.92360000

Epoch: 5/30, Train Loss: 0.28206358, Test Loss: 0.24927074, Test Accuracy: 0.92910000

Epoch: 6/30, Train Loss: 0.26274147, Test Loss: 0.23177691, Test Accuracy: 0.93440000

Epoch: 7/30, Train Loss: 0.24563955, Test Loss: 0.21816125, Test Accuracy: 0.93730000

Epoch: 8/30, Train Loss: 0.22912122, Test Loss: 0.20615057, Test Accuracy: 0.94010000

Epoch: 9/30, Train Loss: 0.21592307, Test Loss: 0.19654794, Test Accuracy: 0.94240000

Epoch: 10/30, Train Loss: 0.20384999, Test Loss: 0.18144420, Test Accuracy: 0.94660000

Epoch: 11/30, Train Loss: 0.19319572, Test Loss: 0.17440862, Test

# Adam

In [ ]:
for i in range(9):
    print(f"Entering {i}-th loop")
    model_Adam = Model(dropout_list[i])
    print("dropout rate:", model_Adam.dropout)
    model_Adam.to(device)
    model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
    optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam)
    
    if i == 0:
        f = open("dropout_sparsity_Adam.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("dropout_sparsity_Adam.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
f.close()

!cp dropout_sparsity_Adam.txt /content/drive/MyDrive

Entering 0-th loop
dropout rate: Dropout(p=0.1, inplace=False)

Epoch: 1/30, Train Loss: 0.44654760, Test Loss: 0.22959960, Test Accuracy: 0.93250000

Epoch: 2/30, Train Loss: 0.20868795, Test Loss: 0.17028411, Test Accuracy: 0.95030000

Epoch: 3/30, Train Loss: 0.15566531, Test Loss: 0.13064775, Test Accuracy: 0.95880000

Epoch: 4/30, Train Loss: 0.12036851, Test Loss: 0.11002178, Test Accuracy: 0.96780000

Epoch: 5/30, Train Loss: 0.09696266, Test Loss: 0.09891282, Test Accuracy: 0.96960000

Epoch: 6/30, Train Loss: 0.08080136, Test Loss: 0.08488593, Test Accuracy: 0.97180000

Epoch: 7/30, Train Loss: 0.06787993, Test Loss: 0.08016853, Test Accuracy: 0.97520000

Epoch: 8/30, Train Loss: 0.05894409, Test Loss: 0.07423516, Test Accuracy: 0.97670000

Epoch: 9/30, Train Loss: 0.04862988, Test Loss: 0.07120875, Test Accuracy: 0.97750000

Epoch: 10/30, Train Loss: 0.04244900, Test Loss: 0.06870488, Test Accuracy: 0.97870000

Epoch: 11/30, Train Loss: 0.03690273, Test Loss: 0.07056477, Test